# Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Foods/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston_TSB algorithm

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [ ]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [ ]:
list_params_alpha = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]
list_params_beta = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [ ]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100
validation_training_ratio = 0.95

# Pipeline for Croston_TSB

In [ ]:
def CrostonTSB_output_all_params(sales_pattern, list_params_alpha, list_params_beta, 
                                 start_train_date, end_train_date, n_pred_days, 
                                 num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params_alpha:
            for j in list_params_beta:
                predictions = list()
                history = [x for x in train_first[num_train_needed*-1:]]
                for t in range(len(train_second)):
                    yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                    predictions.append(yhat)
                    history.append(train_second[t])
                df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                               'Actual Data': train_second,
                                               'Forecast': predictions,
                                               'Alpha': [i for count_i in range(len(train_second))],
                                               'Beta': [j for count_j in range(len(train_second))]})
                df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [ ]:
sales_intermittent

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1617,FOODS_1_006_CA_1_validation,FOODS_1_006,FOODS_1,FOODS,CA_1,CA,0,0,3,1,...,1,3,1,0,2,1,2,3,2,2
1618,FOODS_1_008_CA_1_validation,FOODS_1_008,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12189,FOODS_3_821_CA_4_validation,FOODS_3_821,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,0,1,1,0,3,0,0
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1
12193,FOODS_3_825_CA_4_validation,FOODS_3_825,FOODS_3,FOODS,CA_4,CA,2,3,2,1,...,0,2,0,1,3,1,1,0,2,0


In [ ]:
sales_intermittent.iloc[:500]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1617,FOODS_1_006_CA_1_validation,FOODS_1_006,FOODS_1,FOODS,CA_1,CA,0,0,3,1,...,1,3,1,0,2,1,2,3,2,2
1618,FOODS_1_008_CA_1_validation,FOODS_1_008,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2329,FOODS_3_105_CA_1_validation,FOODS_3_105,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,0,2,1,0,0,1,0,1,2,0
2334,FOODS_3_110_CA_1_validation,FOODS_3_110,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,1,1,2,1,0,0,1,2,1,2
2335,FOODS_3_111_CA_1_validation,FOODS_3_111,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,2,1
2339,FOODS_3_115_CA_1_validation,FOODS_3_115,FOODS_3,FOODS,CA_1,CA,4,0,0,0,...,4,1,2,0,1,1,1,4,1,3


In [ ]:
sales_intermittent.iloc[500:1000]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
2341,FOODS_3_117_CA_1_validation,FOODS_3_117,FOODS_3,FOODS,CA_1,CA,1,1,4,3,...,0,0,0,0,3,1,0,1,2,4
2342,FOODS_3_118_CA_1_validation,FOODS_3_118,FOODS_3,FOODS,CA_1,CA,4,2,6,4,...,4,1,4,0,5,0,2,4,2,3
2343,FOODS_3_119_CA_1_validation,FOODS_3_119,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,1,2,0,0,1,1,0,1,0,0
2346,FOODS_3_122_CA_1_validation,FOODS_3_122,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
2350,FOODS_3_126_CA_1_validation,FOODS_3_126,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,3,4,0,0,1,0,2,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4919,FOODS_2_043_CA_2_validation,FOODS_2_043,FOODS_2,FOODS,CA_2,CA,5,2,3,1,...,2,2,0,1,3,0,1,1,3,4
4921,FOODS_2_045_CA_2_validation,FOODS_2_045,FOODS_2,FOODS,CA_2,CA,0,0,0,0,...,1,0,2,0,1,0,0,1,0,1
4922,FOODS_2_046_CA_2_validation,FOODS_2_046,FOODS_2,FOODS,CA_2,CA,0,0,0,0,...,0,0,1,3,1,0,0,0,2,0
4923,FOODS_2_047_CA_2_validation,FOODS_2_047,FOODS_2,FOODS,CA_2,CA,0,0,0,0,...,3,1,1,0,0,1,0,0,1,1


In [ ]:
sales_intermittent.iloc[1000:1500]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
4925,FOODS_2_049_CA_2_validation,FOODS_2_049,FOODS_2,FOODS,CA_2,CA,0,0,0,0,...,0,0,0,1,0,0,1,1,0,3
4927,FOODS_2_051_CA_2_validation,FOODS_2_051,FOODS_2,FOODS,CA_2,CA,0,0,0,0,...,0,3,4,2,0,2,5,2,1,8
4928,FOODS_2_052_CA_2_validation,FOODS_2_052,FOODS_2,FOODS,CA_2,CA,3,2,2,1,...,1,0,1,1,3,2,1,0,3,3
4929,FOODS_2_053_CA_2_validation,FOODS_2_053,FOODS_2,FOODS,CA_2,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4930,FOODS_2_054_CA_2_validation,FOODS_2_054,FOODS_2,FOODS,CA_2,CA,0,0,0,0,...,3,1,7,2,1,3,0,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5616,FOODS_3_343_CA_2_validation,FOODS_3_343,FOODS_3,FOODS,CA_2,CA,0,0,0,0,...,0,1,0,1,0,0,0,1,1,0
5617,FOODS_3_344_CA_2_validation,FOODS_3_344,FOODS_3,FOODS,CA_2,CA,0,0,0,0,...,1,2,0,1,1,1,1,1,2,4
5619,FOODS_3_346_CA_2_validation,FOODS_3_346,FOODS_3,FOODS,CA_2,CA,6,5,9,6,...,1,2,2,1,4,0,2,0,4,1
5623,FOODS_3_350_CA_2_validation,FOODS_3_350,FOODS_3,FOODS,CA_2,CA,0,0,0,0,...,2,3,2,3,3,2,1,0,1,3


In [ ]:
sales_intermittent.iloc[1500:2000]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
5627,FOODS_3_354_CA_2_validation,FOODS_3_354,FOODS_3,FOODS,CA_2,CA,0,0,0,0,...,0,1,0,1,0,0,1,1,0,4
5629,FOODS_3_356_CA_2_validation,FOODS_3_356,FOODS_3,FOODS,CA_2,CA,1,1,1,0,...,2,1,1,1,4,0,3,0,1,2
5630,FOODS_3_357_CA_2_validation,FOODS_3_357,FOODS_3,FOODS,CA_2,CA,0,0,0,0,...,0,1,0,1,1,0,1,1,3,1
5631,FOODS_3_358_CA_2_validation,FOODS_3_358,FOODS_3,FOODS,CA_2,CA,0,0,0,0,...,0,1,1,1,0,2,0,0,0,0
5632,FOODS_3_359_CA_2_validation,FOODS_3_359,FOODS_3,FOODS,CA_2,CA,0,0,0,0,...,0,0,2,0,2,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8126,FOODS_2_202_CA_3_validation,FOODS_2_202,FOODS_2,FOODS,CA_3,CA,0,2,0,1,...,1,0,2,0,0,0,1,2,1,0
8130,FOODS_2_206_CA_3_validation,FOODS_2_206,FOODS_2,FOODS,CA_3,CA,0,0,0,0,...,2,1,2,0,3,2,0,1,0,2
8131,FOODS_2_207_CA_3_validation,FOODS_2_207,FOODS_2,FOODS,CA_3,CA,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
8132,FOODS_2_208_CA_3_validation,FOODS_2_208,FOODS_2,FOODS,CA_3,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0


In [ ]:
sales_intermittent.iloc[2000:2500]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
8135,FOODS_2_211_CA_3_validation,FOODS_2_211,FOODS_2,FOODS,CA_3,CA,1,2,1,2,...,1,0,2,0,1,0,1,0,2,0
8136,FOODS_2_212_CA_3_validation,FOODS_2_212,FOODS_2,FOODS,CA_3,CA,3,3,1,3,...,2,1,7,3,4,3,1,4,4,5
8140,FOODS_2_216_CA_3_validation,FOODS_2_216,FOODS_2,FOODS,CA_3,CA,0,0,0,0,...,2,3,0,3,0,1,0,3,0,0
8141,FOODS_2_217_CA_3_validation,FOODS_2_217,FOODS_2,FOODS,CA_3,CA,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
8142,FOODS_2_218_CA_3_validation,FOODS_2_218,FOODS_2,FOODS,CA_3,CA,1,0,1,0,...,1,2,3,3,2,2,0,3,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9132,FOODS_3_813_CA_3_validation,FOODS_3_813,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,3,4,8,2,2,2,1,0,5,5
9133,FOODS_3_814_CA_3_validation,FOODS_3_814,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,2,0,1,8,0,1,2,0,0,5
9136,FOODS_3_817_CA_3_validation,FOODS_3_817,FOODS_3,FOODS,CA_3,CA,1,0,0,1,...,0,1,0,0,2,0,1,0,1,2
9140,FOODS_3_821_CA_3_validation,FOODS_3_821,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,6,0,3,1,1,3,0,0,0,3


In [ ]:
sales_intermittent.iloc[2500:3000]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
9144,FOODS_3_825_CA_3_validation,FOODS_3_825,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,3,0,4,3,3,0,0,3,3,6
9145,FOODS_3_826_CA_3_validation,FOODS_3_826,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,5,2,2,0,1,1,3,0,1,3
10759,FOODS_1_001_CA_4_validation,FOODS_1_001,FOODS_1,FOODS,CA_4,CA,0,1,1,1,...,0,0,0,2,0,0,0,1,1,1
10760,FOODS_1_002_CA_4_validation,FOODS_1_002,FOODS_1,FOODS,CA_4,CA,0,1,2,1,...,0,0,0,0,0,0,0,0,0,0
10761,FOODS_1_003_CA_4_validation,FOODS_1_003,FOODS_1,FOODS,CA_4,CA,0,1,0,0,...,1,0,0,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11370,FOODS_2_397_CA_4_validation,FOODS_2_397,FOODS_2,FOODS,CA_4,CA,1,0,1,0,...,1,2,2,1,1,2,0,0,0,0
11371,FOODS_2_398_CA_4_validation,FOODS_2_398,FOODS_2,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,0,2,0,0,0,0,1
11372,FOODS_2_399_CA_4_validation,FOODS_2_399,FOODS_2,FOODS,CA_4,CA,6,7,3,3,...,0,3,0,4,1,0,0,0,4,7
11373,FOODS_3_001_CA_4_validation,FOODS_3_001,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0


In [ ]:
sales_intermittent.iloc[3000:]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
11375,FOODS_3_003_CA_4_validation,FOODS_3_003,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,2,1,0,0,0,0,1,0
11376,FOODS_3_004_CA_4_validation,FOODS_3_004,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,1,0,0,0,0,2,0
11377,FOODS_3_005_CA_4_validation,FOODS_3_005,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11378,FOODS_3_006_CA_4_validation,FOODS_3_006,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,1,0,1,0,0,1,1,0,2
11381,FOODS_3_009_CA_4_validation,FOODS_3_009,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,2,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12189,FOODS_3_821_CA_4_validation,FOODS_3_821,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,0,1,1,0,3,0,0
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1
12193,FOODS_3_825_CA_4_validation,FOODS_3_825,FOODS_3,FOODS,CA_4,CA,2,3,2,1,...,0,2,0,1,3,1,1,0,2,0


In [ ]:
start = timer()

pattern_df_result_all_params = CrostonTSB_output_all_params(sales_intermittent.iloc[3000:],
                                                          list_params_alpha,
                                                          list_params_beta,
                                                          start_train_date,
                                                          end_train_date, 
                                                          n_pred_days,
                                                          num_train_needed,
                                                          validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 21.089674221816647 minutes


In [ ]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,FOODS_3_003_CA_4_validation,0.0,0.000000,0.1,0.1
1,FOODS_3_003_CA_4_validation,0.0,0.000000,0.1,0.1
2,FOODS_3_003_CA_4_validation,0.0,0.000000,0.1,0.1
3,FOODS_3_003_CA_4_validation,0.0,0.000000,0.1,0.1
4,FOODS_3_003_CA_4_validation,0.0,0.000000,0.1,0.1
...,...,...,...,...,...
1741252,FOODS_3_826_CA_4_validation,4.0,0.998324,0.9,0.9
1741253,FOODS_3_826_CA_4_validation,4.0,3.674065,0.9,0.9
1741254,FOODS_3_826_CA_4_validation,1.0,3.967149,0.9,0.9
1741255,FOODS_3_826_CA_4_validation,2.0,1.296980,0.9,0.9


In [ ]:
pattern_df_result_all_params.to_csv('CrostonTSB_Intermittent_3000_End_All_Parameters_Hyperparameter_Tuning_Data.csv')

In [ ]:
df_res_1 = pd.read_csv('/content/CrostonTSB_Intermittent_0_500_All_Parameters_Hyperparameter_Tuning_Data.csv', index_col=0)
df_res_2 = pd.read_csv('/content/CrostonTSB_Intermittent_500_1000_All_Parameters_Hyperparameter_Tuning_Data.csv', index_col=0)
df_res_3 = pd.read_csv('/content/CrostonTSB_Intermittent_1000_1500_All_Parameters_Hyperparameter_Tuning_Data.csv', index_col=0)
df_res_4 = pd.read_csv('/content/CrostonTSB_Intermittent_1500_2000_All_Parameters_Hyperparameter_Tuning_Data.csv', index_col=0)
df_res_5 = pd.read_csv('/content/CrostonTSB_Intermittent_2000_2500_All_Parameters_Hyperparameter_Tuning_Data.csv', index_col=0)
df_res_6 = pd.read_csv('/content/CrostonTSB_Intermittent_2500_3000_All_Parameters_Hyperparameter_Tuning_Data.csv', index_col=0)
df_res_7 = pd.read_csv('/content/CrostonTSB_Intermittent_3000_End_All_Parameters_Hyperparameter_Tuning_Data.csv', index_col=0)

In [ ]:
pattern_df_result_all_params = pd.concat([df_res_1, df_res_2, df_res_3, df_res_4, df_res_5, df_res_6, df_res_7], axis=0)

In [1]:
pattern_df_result_all_params

NameError: ignored

# Summay all metrics

In [ ]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 8.472060987699994 minutes


In [ ]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_001_CA_1_validation,0.1,0.1,0.870489,127.8098,160.0361,56.0841,0.967210,1.655044
1,FOODS_1_001_CA_1_validation,0.1,0.2,0.875894,128.6035,160.3922,55.3300,0.973216,1.693323
2,FOODS_1_001_CA_1_validation,0.1,0.3,0.878565,128.9957,161.3187,55.6299,0.976183,1.724640
3,FOODS_1_001_CA_1_validation,0.1,0.4,0.874199,128.3546,162.2295,55.4020,0.971332,1.749631
4,FOODS_1_001_CA_1_validation,0.1,0.5,0.873650,128.2740,164.0613,57.7256,0.970722,1.770915
...,...,...,...,...,...,...,...,...,...
290056,FOODS_3_826_CA_4_validation,0.9,0.5,0.958268,85.9822,89.5803,59.8554,1.464021,2.006461
290057,FOODS_3_826_CA_4_validation,0.9,0.6,0.947464,85.0127,90.6077,60.5278,1.447514,2.008553
290058,FOODS_3_826_CA_4_validation,0.9,0.7,0.935267,83.9184,91.9283,61.0804,1.428880,2.016023
290059,FOODS_3_826_CA_4_validation,0.9,0.8,0.941459,84.4739,95.0862,63.2735,1.438340,2.030263


# Check what products has unexpected metrics result

In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
810,FOODS_1_008_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
811,FOODS_1_008_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
812,FOODS_1_008_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
813,FOODS_1_008_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
814,FOODS_1_008_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...
289975,FOODS_3_826_CA_3_validation,0.9,0.5,inf,inf,200.0,NaN,2.805754e-10,9.853484e-10
289976,FOODS_3_826_CA_3_validation,0.9,0.6,inf,inf,200.0,NaN,3.649518e-13,1.453276e-12
289977,FOODS_3_826_CA_3_validation,0.9,0.7,inf,inf,200.0,NaN,7.469419e-17,3.333996e-16
289978,FOODS_3_826_CA_3_validation,0.9,0.8,inf,inf,200.0,NaN,5.115764e-22,2.540772e-21


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
810,FOODS_1_008_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
811,FOODS_1_008_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
812,FOODS_1_008_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
813,FOODS_1_008_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
814,FOODS_1_008_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...
289975,FOODS_3_826_CA_3_validation,0.9,0.5,inf,inf,200.0,NaN,2.805754e-10,9.853484e-10
289976,FOODS_3_826_CA_3_validation,0.9,0.6,inf,inf,200.0,NaN,3.649518e-13,1.453276e-12
289977,FOODS_3_826_CA_3_validation,0.9,0.7,inf,inf,200.0,NaN,7.469419e-17,3.333996e-16
289978,FOODS_3_826_CA_3_validation,0.9,0.8,inf,inf,200.0,NaN,5.115764e-22,2.540772e-21


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
810,FOODS_1_008_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
811,FOODS_1_008_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
812,FOODS_1_008_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
813,FOODS_1_008_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
814,FOODS_1_008_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
289165,FOODS_3_823_CA_1_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
289166,FOODS_3_823_CA_1_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
289167,FOODS_3_823_CA_1_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
289168,FOODS_3_823_CA_1_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
810,FOODS_1_008_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
811,FOODS_1_008_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
812,FOODS_1_008_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
813,FOODS_1_008_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
814,FOODS_1_008_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...
289975,FOODS_3_826_CA_3_validation,0.9,0.5,inf,inf,200.0,NaN,2.805754e-10,9.853484e-10
289976,FOODS_3_826_CA_3_validation,0.9,0.6,inf,inf,200.0,NaN,3.649518e-13,1.453276e-12
289977,FOODS_3_826_CA_3_validation,0.9,0.7,inf,inf,200.0,NaN,7.469419e-17,3.333996e-16
289978,FOODS_3_826_CA_3_validation,0.9,0.8,inf,inf,200.0,NaN,5.115764e-22,2.540772e-21


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [ ]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [ ]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 31.778832728288187 %
Percentage of unexpected values of WMAPE is: 31.778832728288187 %
Percentage of unexpected values of SMAPE is: 30.717676626640607 %
Percentage of unexpected values of MAPE is: 31.778832728288187 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [ ]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE       WMAPE       SMAPE       MAPE       MAE      RMSE
Alpha Beta                                                                 
0.1   0.1   0.930741  120.586433  133.861666  58.731577  1.128197  1.475853
      0.2   0.913144  117.909089  134.703624  60.054045  1.126227  1.485485
      0.3   0.909814  117.407323  135.852256  61.529229  1.130639  1.501546
      0.4   0.910553  117.457745  137.220254  63.187955  1.137899  1.520807
      0.5   0.913957  117.863647  138.830704  65.111876  1.147634  1.542875
...              ...         ...         ...        ...       ...       ...
0.9   0.5   0.951359  121.252226  143.053707  72.210658  1.242592  1.700616
      0.6   0.956476  121.900919  144.584558  74.087469  1.253859  1.729680
      0.7   0.963388  122.786749  146.375371  76.176035  1.267148  1.762426
      0.8   0.972592  123.957920  148.557714  78.586884  1.283282  1.799595
      0.9   0.983556  125.345837  151.305807  81.301579  1.302076  1.842188

[81 rows x 6 columns]

In [ ]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.1, 0.3)
The optimum Alpha and Beta based on Mean WMAPE is: (0.2, 0.3)
The optimum Alpha and Beta based on Mean SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAE is: (0.1, 0.2)
The optimum Alpha and Beta based on Mean RMSE is: (0.1, 0.1)


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE     WMAPE     SMAPE     MAPE       MAE      RMSE
Alpha Beta                                                           
0.1   0.1   0.855367  103.8600  130.7446  53.7194  0.881148  1.148728
      0.2   0.859387  104.7108  132.8207  54.8155  0.896194  1.166628
      0.3   0.864818  105.8467  135.1981  56.1450  0.909546  1.188680
      0.4   0.871734  106.7552  137.1678  58.3693  0.919790  1.209226
      0.5   0.879110  107.9468  139.6393  60.5345  0.933930  1.231262
...              ...       ...       ...      ...       ...       ...
0.9   0.5   0.936008  113.0208  144.3837  69.2129  0.977599  1.326570
      0.6   0.944235  114.2983  146.4481  71.5506  0.987359  1.359323
      0.7   0.952987  115.4787  149.2360  73.8294  0.999589  1.393546
      0.8   0.963079  117.1344  152.5719  76.6663  1.017334  1.425279
      0.9   0.972973  118.5246  156.2093  79.7654  1.032668  1.461896

[81 rows x 6 columns]

In [ ]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of WMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of RMSE is: (0.1, 0.1)


# Run CrostonTSB model after decide best paramaters

In [ ]:
best_alpha = 0.1
best_beta = 0.1

In [ ]:
def CrostonTSB_test_data(best_alpha, best_beta, sales_pattern,
                         start_train_date, end_train_date, 
                         n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [ ]:
start = timer()

pattern_df_result_best_params = CrostonTSB_test_data(best_alpha, best_beta, sales_intermittent, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.7502668759499997 minutes


In [ ]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [ ]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_001_CA_1_validation,0.796917,137.3149,157.0766,54.2178,0.980821,1.212029
1,FOODS_1_001_CA_4_validation,0.847988,136.9826,179.7702,77.7376,0.782758,1.330052
2,FOODS_1_002_CA_1_validation,0.673822,145.1310,199.5511,98.4038,0.310995,0.796817
3,FOODS_1_002_CA_3_validation,1.412904,152.1589,188.0671,83.6249,0.326055,0.424292
4,FOODS_1_002_CA_4_validation,0.830259,111.7656,164.0421,72.1531,0.957991,1.212305
...,...,...,...,...,...,...,...
3576,FOODS_3_825_CA_4_validation,2.485424,535.3221,194.5902,76.6435,0.382373,0.405992
3577,FOODS_3_826_CA_1_validation,1.066387,100.4864,114.0111,94.8040,2.296833,2.553122
3578,FOODS_3_826_CA_2_validation,0.805441,108.4248,111.3365,18.5303,0.619570,0.759575
3579,FOODS_3_826_CA_3_validation,inf,inf,200.0000,NaN,0.001168,0.001266


In [ ]:
df_result_final.to_csv('CrostonTSB_Intermittent_Test_Data.csv')